In [17]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
#import unicodecsv
import re
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import gensim

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
data = pd.read_csv('amazon_reviews.txt', sep="\t",  error_bad_lines=False)

In [34]:
data.head()

,DOC_ID,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,1,__label1__,4,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,2,__label1__,4,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...
2,3,__label1__,3,N,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...
3,4,__label1__,4,N,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...
4,5,__label1__,4,N,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...


In [0]:
x_raw = data['REVIEW_TEXT']
x_raw = data['REVIEW_TEXT'].values.tolist()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  

In [0]:
x_raw = [a.replace("<br />", "") for a in x_raw]

In [0]:
#Preprocess corpus

from string import punctuation
from nltk.corpus import stopwords
import string

def preprocess(text):
    # word tokenisation, including punctuation removal
    tokenizer = RegexpTokenizer(r'\w+')

    #text = str(text)
    #tokens = tokenizer.tokenize(text)
    tokens = text
    
    # lowercasing
    tokens = [t.lower() for t in tokens]

    # stopword removal
    stop = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop]
    
    # lemmatisation
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(t) for t in tokens]


    digits = ' 0123456789'
    tokens = [t for t in tokens if t not in digits]
    
    return tokens

In [0]:
x = preprocess(x_raw)

In [0]:
x = list(sent_to_words(x))

In [40]:
bigram = gensim.models.Phrases(x, min_count=3, threshold=30) # higher threshold fewer phrases.
#bigram_mod = gensim.models.phrases.Phraser(bigram)

#def make_bigrams(texts):
#    return [bigram_mod[doc] for doc in texts]

#x = make_bigrams(bigram)
x = [bigram[line] for line in x]

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [41]:
x = [' '.join(i) for i in x]
print(x[:100])

['when least you think so this product will save the day just keep it around just in case you need it for something', 'lithium_batteries are something new introduced in the market there average developing cost is relatively high but stallion doesn compromise on quality and provides us with the best at low_cost there are so many in built technical assistants that act like sensor in their particular forte the battery keeps my phone charged up and it works at every voltage and high voltage is never risked', 'purchased this swing for my baby she is months now and has pretty much out grown it it is very loud and doesn swing very well it is beautiful though love the colors and it has lot of settings but don think it was worth the money', 'was looking for an inexpensive desk calcolatur and here it is it works and does everything need only issue is that it tilts slightly to one side so when hit any keys it rocks little bit not big_deal', 'only use it twice_week and the results are great have u

In [27]:
#Apply LDA to corpus
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


count_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
x_train = count_vect.fit_transform(x)

LDA = LatentDirichletAllocation(n_components=10, learning_decay=0.5, random_state=42)
LDA.fit(x_train)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.5,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [28]:
def show_topics(vectorizer=count_vect, lda_model=LDA, n_words=5):
    keywords = np.array(count_vect.get_feature_names())
    topic_keywords = []
    for topic_weights in LDA.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=count_vect, lda_model=LDA, n_words=5)        

print('Top 5 words for each topic')
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

Top 5 words for each topic


,Word 0,Word 1,Word 2,Word 3,Word 4
Topic 0,coffee,tea,love,taste,skin
Topic 1,skin,hair,great,shampoo,product
Topic 2,great,good,just,use,product
Topic 3,glasses,jeans,polarized,hood,taste
Topic 4,great,good,product,like,love
Topic 5,wii,local_stores,black_friday,couldn_wait,gamepad
Topic 6,great,love,size,really,good
Topic 7,game,great,book,love,good
Topic 8,shoes,great,knife,love,shoe
Topic 9,book,story,read,movie,film


In [0]:
!pip install pyLDAvis
from pyLDAvis import sklearn as sklearn_lda
import os
import pickle 
import pyLDAvis
LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(10))

if 1 == 1:
    LDAvis_prepared = sklearn_lda.prepare(LDA, x_train, count_vect)
with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
     LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(10) +'.html')

pyLDAvis.display(LDAvis_prepared)

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 17.0MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=9fd912b2ee373c7b0be55b6cf13bd65093fecdacee9d537959cb4cbb44605fbb
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=b67c6d4f4c263efcdfaba12e53dcc640ca8082aca00b64f226ee23a1fc76f6fd
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [29]:
# Topic ID for each review in dataset: review ID, review text, topic ID

topic_values = LDA.transform(x_train)
data['Topic'] = topic_values.argmax(axis=1)
#data[['REVIEW_TEXT', 'Topic']]

result = data[['REVIEW_TEXT', 'Topic']][:100]
from IPython.display import display, HTML

display(HTML(result.to_html()))

,REVIEW_TEXT,Topic
0,"When least you think so, this product will save the day. Just keep it around just in case you need it for something.",2
1,Lithium batteries are something new introduced in the market there average developing cost is relatively high but Stallion doesn't compromise on quality and provides us with the best at a low cost.<br />There are so many in built technical assistants that act like a sensor in their particular forté. The battery keeps my phone charged up and it works at every voltage and a high voltage is never risked.,2
2,"I purchased this swing for my baby. She is 6 months now and has pretty much out grown it. It is very loud and doesn't swing very well. It is beautiful though. I love the colors and it has a lot of settings, but I don't think it was worth the money.",2
3,I was looking for an inexpensive desk calcolatur and here it is. It works and does everything I need. Only issue is that it tilts slightly to one side so when I hit any keys it rocks a little bit. Not a big deal.,2
4,"I only use it twice a week and the results are great. I have used other teeth whitening solutions and most of them, for the same results I would have to use it at least three times a week. Will keep using this because of the potency of the solution and also the technique of the trays, it keeps everything in my teeth, in my mouth.",7
5,I'm not sure what this is supposed to be but I would recommend that you do a little more research into the culture of using pipes if you plan on giving this as a gift or using it yourself.,7
6,"Pleased with ping pong table. 11 year old and 13 year old having a blast, plus lots of family entertainment too. Plus better than kids sitting on video games all day. A friend put it together. I do believe that was a challenge, but nothing they could not handle",6
7,"Great vitamin C serum... I really like the oil feeling, not too sticky. I used it last week on some of my recent bug bites and it helps heal the skin faster than normal.",7
8,"I've used tide pods laundry detergent for many years,its such a great detergent to use having a nice scent and leaver the cloths smelling fresh.",4
9,"Everybody wants to fall for their promises. But this is a relatively unheard of brand, some even say a non existant company. Look at how amateur their labels and products are. You have to ask yourself if you would trust this kind of amateur stuff? No way! Don't waste your money.",2


In [32]:
nmf = NMF(n_components=10, random_state=42)
nmf.fit(x_train)

def show_topics2(vectorizer=count_vect, lda_model=nmf, n_words=5):
    keywords = np.array(count_vect.get_feature_names())
    topic_keywords = []
    for topic_weights in nmf.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
    
topic_keywords2 = show_topics2(vectorizer=count_vect, lda_model=nmf, n_words=5)        

print('Top 5 words for each topic')
# Topic - Keywords Dataframe
df_topic_keywords2 = pd.DataFrame(topic_keywords2)
df_topic_keywords2.columns = ['Word '+str(i) for i in range(df_topic_keywords2.shape[1])]
df_topic_keywords2.index = ['Topic '+str(i) for i in range(df_topic_keywords2.shape[0])]
df_topic_keywords2

Top 5 words for each topic


,Word 0,Word 1,Word 2,Word 3,Word 4
Topic 0,just,time,got,work,bought
Topic 1,product,recommend,happy,excellent,buy
Topic 2,watch,looks,band,time,watches
Topic 3,good,quality,price,sound,cheap
Topic 4,size,fit,perfect,nice,small
Topic 5,love,beautiful,color,kids,buy
Topic 6,use,easy,works,light,clean
Topic 7,great,price,works,quality,looks
Topic 8,really,like,looks,look,feel
Topic 9,bag,carry,leather,bags,inside
